Setup a new notebook / .py file and load in the data for your chosen 10-15 stocks AND the market portfolio*. Then...

In [1]:
# import libraries
import pandas as pd # for data management and analysis
import numpy as np # for numerical computations
import matplotlib.pyplot as plt #for plotting
import seaborn as sns # to make charts look good
sns.set() # implement seaborn plotting templates/theme
from scipy.optimize import minimize # for optimisation framework

In [21]:
#import data
df10 = pd.read_csv('https://raw.githubusercontent.com/kpace1111/portfoliomanagement/main/10stocks_assignment.csv')
df10 = df10.loc[:, ~df10.columns.str.contains('^Unnamed')] #remove unamed columns
df10['date'] = pd.to_datetime(df10['date']) # create dates as timestamp
df10.set_index('date', inplace=True)
#df10.head()
returns_df10 = df10.pct_change(1).dropna() #calc returns of all 10 stocks
#returns_df10

In [22]:
#import data
dfsp500 = pd.read_csv('https://raw.githubusercontent.com/kpace1111/portfoliomanagement/main/us500_price.csv')
dfsp500['date'] = pd.to_datetime(dfsp500['date']) # create dates as timestamp
dfsp500.set_index('date', inplace=True)
#dfsp500.head()

- Create a vector of weights which sum up to 1.

In [33]:
num_stocks = len(returns_df10.columns) 
init_weights = [1/num_stocks] * num_stocks #vector of weights

- Calculate the annualised Expected Return of the portfolio as it is.

In [24]:
# using scipy minimise method
#create function too calc returns
def getPortReturn(weights):
    exp_ret_portfolio = np.dot(np.transpose(weights), returns_df10.mean()) * 250 #annualise crude method
    
    return exp_ret_portfolio

getPortReturn(init_weights)

0.15937560313247898

- Calculate the annualised Standard Deviation of the portfolio as it is.

In [61]:
vcv_matrix = returns_df10.cov() # sigma - works out var and cov for all 10 stocks

# take transposed of vector of weights and multiply by product of the VCV matrix and the vector of weights 
var_p = np.dot(np.transpose(weights), np.dot(vcv_matrix, weights)) #variance of 10 asset portfolio
sd_p = np.sqrt(var_p) #daily SD of 10 asset portfolio
sd_p_annual = sd_p * np.sqrt(250)

sd_p_annual

0.23019518687128937

- Calculate the annualised Expected Return and Standard Deviation for the market portfolio.

In [34]:
## create a function to calcualate annualised expected returns

def getExpectedReturn(df, price_col_name, annualised=True, annualise_method='sophisticated'):
    """
    Returns the expected return of a stock given price data
    """
    
    #Calculate returns of prices, stores them under returns
    returns = df[price_col_name].pct_change(1)
    
    #Calculate the expected return using the mean method
    expected_return_daily = returns.mean()
    
    if annualised:
        if annualise_method == 'sophisticated':
            expected_return_annual = ((1 + expected_return_daily) ** 250) -1
        elif annualise_method == 'crude':
            # crude method
            expected_return_annual = expected_return_daily * 250
        return expected_return_annual
    else:
        return expected_return_daily
    

getExpectedReturn(df=dfsp500, price_col_name='price', annualise_method='sophisticated')

0.09803988805644015

In [35]:
dfsp500['returns'] = dfsp500['price'].pct_change(1) #calculate returns
sd_sp500 = np.std(dfsp500['returns'], ddof=1)
sd_sp500_annual = sd_sp500 * np.sqrt(250)
sd_sp500_annual

0.20364418439412446

# Questions for this assignment # 




## 1. Optimise your portfolio weights so that the expected return on your 10-15 asset portfolio is equal to the annualised expected return on the market portfolio. What is the risk of your portfolio now? How does it compare to the market portfolio? How does it compare to the performance using the initialised weights? ##

In [68]:
target_return = 0.098
bounds = tuple((0, 1) for i in range (num_stocks)) # for loop, create tuple of 0, 1 for every stock
cons = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) -1 }, #create constraint with tuple with 2 dictionaries inside, with each relating to a contraint in the minimise method, when the function inside contrant = 0, it has satisfied constraint
        {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df10.mean()) * 250 - target_return }) #second constraint, finds the difference between the E(rp) and the target must = 0 by minimising function (=0) and satisifying constraint

results = minimize(fun=getPortReturn, x0=init_weights, bounds=bounds, constraints=cons)

optimised_weights = pd.DataFrame(results['x']) # create df for the optimised weights
optimised_weights.index = returns_df10.columns # add tickers as index
optimised_weights #high risk, high reward

,0
msft_price,2.624833e-02
amzn_price,8.708099e-02
googl_price,9.997273e-02
aapl_price,2.775558e-17
fb_price,2.584180e-01
brkb_price,1.328736e-01
mcd_price,1.239360e-01
hd_price,1.002538e-01
xom_price,1.349084e-01
unh_price,3.630811e-02


In [69]:
getPortReturn(weights=results['x']) # pull optimised weights from above, confirming result

0.09799999976407359

In [70]:
def getPortRisk(weights):
    
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    returns_df10 = df10.pct_change(1).dropna()  # estimate returns for each asset
    num_stocks = len(returns_df10.columns)  # number of stocks based on number of columns (excluding index col)
                                          # this is a local variable
        
    vcv = returns_df10.cov()  # being the variance covariance matrix
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))  # variance of the multi-asset portfolio
    sd_p = np.sqrt(var_p)  # standard deviation of the multi-asset portfolio
    sd_p_annual = sd_p * np.sqrt(250)  # annualised standard deviation of the multi-asset portfolio
    
    return sd_p_annual

In [71]:
getPortRisk(results['x'])

0.23926564285480606

The original portfolio return was about 16%, when we decreased this target return to be 9.8% like the market portfolio, standard deviation actually increased, but very slightly from 23% to 24%, this was not really expected as risk and return are positively correlated. Overall this portfolio has higher risk and performs worse than the original portfolio. 

## 2. Optimise your portfolio weights so that the risk of your 10-15 asset portfolio is equal to the annualised risk of the market portfolio. What is the expected return on your portfolio now? How does it compare to the market portfolio?

In [74]:
target_risk = 0.2
num_stocks = len(df10.columns)  # being the number of stocks (this is a 'global' variable)
init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0)

bounds = tuple((0, 1) for i in range(num_stocks))

cons = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1}, #create constraint with tuple with 2 dictionaries inside, with each relating to a contraint in the minimise method, when the function inside contrant = 0, it has satisfied constraint
        {'type' : 'eq', 'fun' : lambda x: np.sqrt(np.dot(x, np.dot(returns_df10.cov(), np.transpose(x))))*np.sqrt(250) - target_risk}) #second constraint, finds the difference between the E(rp) and the target must = 0 by minimising function (=0) and satisifying constraint

results = minimize(fun=getPortReturn, x0=init_weights, bounds=bounds, constraints=cons)

optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = df10.columns
optimised_weights.rename(columns={optimised_weights.columns[0] : 'weights'}, inplace=True)
optimised_weights['weights_rounded'] = optimised_weights['weights'].apply(lambda x : round(x, 3))
optimised_weights['weights_rounded'].sort_values(ascending=False).cumsum()

mcd_price      0.372
brkb_price     0.737
amzn_price     0.866
unh_price      0.931
xom_price      0.980
hd_price       1.000
msft_price     1.000
googl_price    1.000
aapl_price     1.000
fb_price       1.000
Name: weights_rounded, dtype: float64

In [75]:
#new expected return
getPortReturn(results['x'])

0.1357914262496865

As we decreased the target risk from 23% to 20%, the target returns also reduced from 16% to 13.6%. This was expected as when risk decreases so should returns. 

## 3. Optimise your portfolio weights so that the risk of your portfolio is minimised. What is the expected return on your portfolio now? How does it compare to the market portfolio?

In [76]:
bounds = tuple((0, 1) for i in range(num_stocks))
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

In [77]:
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)
results

     fun: 0.20246113598576024
     jac: array([0.22047493, 0.20255713, 0.20625286, 0.22069   , 0.21775961,
       0.20234111, 0.20243664, 0.20220008, 0.2024904 , 0.20309921])
 message: 'Optimization terminated successfully'
    nfev: 143
     nit: 13
    njev: 13
  status: 0
 success: True
       x: array([1.43313320e-17, 1.29557802e-01, 0.00000000e+00, 2.82907427e-18,
       1.60894437e-17, 3.63128735e-01, 3.72072702e-01, 1.79077045e-02,
       4.91539440e-02, 6.81791136e-02])

In [80]:
# Explore optimised weights
optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = df10.columns
optimised_weights.rename(columns={optimised_weights.columns[0] : 'weights'}, inplace=True)
optimised_weights['weights_rounded'] = optimised_weights['weights'].apply(lambda x : round(x, 3))
optimised_weights['weights_rounded'].sort_values(ascending=False).cumsum()

mcd_price      0.372
brkb_price     0.735
amzn_price     0.865
unh_price      0.933
xom_price      0.982
hd_price       1.000
msft_price     1.000
googl_price    1.000
aapl_price     1.000
fb_price       1.000
Name: weights_rounded, dtype: float64

In [83]:
getPortRisk(results['x'])

0.20246113598576024

In [81]:
getPortReturn(weights=results['x']) #pull optimised weights from above, confirming result

0.13609681282456434

The expected return is 13.6% which is still higher than the return from the market portfolio.

# 4. Based on the results from your optimisations, would you invest in your portfolio, or would you invest in the market portfolio instead? Why?

When the 10 stock portfolio's risk is minimised, it has a SD of around 20.2% and an expected return of 13.6%, the market portfolio has a SD of 20.3% and a return of 9.8%. Just by this alone I would invest in the portfolio rather than the market portfolio. 